# Experiment 1:
## Approach: 
* Universal Sentence Encoder Large 
* Template matrix build from text encodings 
* No text preprocesseing
* Similarity measured between template and document.
* Similarity below 0.7 ruled 'other.

## Dataset:
* interim_output_template_500_documents_rev4.xlsx.
* 476 documents tested (some not included due to textract problem).
* 19 templates used - [acdbcf, ahwcf, ahwcf_v3, ahwcf_v4, aicf_pg1, aicf_pg2, aicf_v1, aicf_v2, aicf_v3, canscr, clmapp, hicf_pg1, hicf_pg2, init_pg1_v2, init_pg3, phystmt, ptscf, pvbcf]
* Available [here]("s3://aaca-ani-experiments-data/aaca-docdig-test/offline_cf_classification/templates/template_images/")

# Result: 97% F1 Score

In [ ]:
#!pip install -r requirements.txt

In [4]:
%run create_template_main

01/03/2023 12:16:51 | __main__ | INFO: Beginning creation of templates
01/03/2023 12:18:08 | __main__ | INFO: Template creation complete


In [ ]:
%run run_get_text_for_test.py

In [2]:
run run_on_test_set.py

2022-11-03 14:38:23.099108: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-11-03 14:38:23.099147: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2022-11-03 14:38:27.108675: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-11-03 14:38:27.108715: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2022-11-03 14:38:27.108746: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (sagemaker-data-scienc-ml-t3-medium-5812005f1de07e20cb211bb2dcf1): /proc/driver/nvidia/version does not exist
2022-11-03 14

11/03/2022 14:38:53 | __main__ | INFO: Beginning classification run on test set: aaca-docdig-test/offline_cf_classification/test_sets/text_files_500/
11/03/2022 14:42:07 | __main__ | INFO: Completed classification run
11/03/2022 14:42:07 | __main__ | INFO: Results saved to: data/results/experiment_1.xlsx


In [6]:
import boto3
from sklearn.metrics import classification_report
from utils import fix_others

In [4]:
df = pd.read_excel('data/results/experiment_1.xlsx', index_col=0, engine='openpyxl')
df.head()

,json,template,textarct_key_value_dictionary,text,png_path,template_from_interim_logic,Unnamed: 7,revised_template,results,score
0,doc-digitization-pipeline/AD/P0X024W1_02751999...,ahwcf,"{'ZIP:': '3 2 5 0 4', 'Primary Pollcyholder': ...",02-26-21:11:174M; ;11 # 2/ 2 20 ACCIDENT WELLN...,doc-digitization-pipeline/AD/P0X024W1_02751999...,ahwcf,NaN,ahwcf_v3,ahwcf_v3,0.856593
1,doc-digitization-pipeline/AD/PX372696_02641791...,aicf_pg2,"{'*Date of Birth (mm/dd/yy)': '/ /', 'DATE': '...",PX372696 Policyholder Information: *Last Name ...,doc-digitization-pipeline/AD/PX372696_02641791...,aicf_pg2,NaN,aicf_pg2,aicf_pg2,0.917460
2,doc-digitization-pipeline/AD/P0342280_02744692...,aicf_v1,"{'State': 'IL', 'Initial': 'R.', 'ZIP': '61234...",ACCIDENTAL INJURY CLAIM FORM Failure to comple...,doc-digitization-pipeline/AD/P0342280_02744692...,other,NaN,aicf_v1,aicf_v1,0.911419
3,doc-digitization-pipeline/AD/P0L700T5_02844807...,aicf_pg2,"{'No': 'X', '*First Name': 'M I C H A E L', 'D...",if you have additional bills or medical docume...,doc-digitization-pipeline/AD/P0L700T5_02844807...,aicf_pg2,NaN,aicf_pg2,aicf_pg2,0.931417
4,doc-digitization-pipeline/AD/P0P9L5Y4_02716028...,aicf_pg2,"{'""Date of Birth (mm/dd/yy)': 'D 8 / I 8 / 0 6...",02-10-21:02:42PM; AFLAC Claims ;7149869600 3/ ...,doc-digitization-pipeline/AD/P0P9L5Y4_02716028...,aicf_pg2,NaN,aicf_pg2,ptscf,0.781522


In [5]:
filtered_df = df[df['results'] != -100]

In [6]:
filtered_df['results'] = filtered_df.apply(lambda row: fix_others(row), axis=1)

/tmp/ipykernel_20/1466064053.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df['results'] = filtered_df.apply(lambda row: fix_others(row), axis=1)


In [7]:
filtered_df.head()

,json,template,textarct_key_value_dictionary,text,png_path,template_from_interim_logic,Unnamed: 7,revised_template,results,score
0,doc-digitization-pipeline/AD/P0X024W1_02751999...,ahwcf,"{'ZIP:': '3 2 5 0 4', 'Primary Pollcyholder': ...",02-26-21:11:174M; ;11 # 2/ 2 20 ACCIDENT WELLN...,doc-digitization-pipeline/AD/P0X024W1_02751999...,ahwcf,NaN,ahwcf_v3,ahwcf_v3,0.856593
1,doc-digitization-pipeline/AD/PX372696_02641791...,aicf_pg2,"{'*Date of Birth (mm/dd/yy)': '/ /', 'DATE': '...",PX372696 Policyholder Information: *Last Name ...,doc-digitization-pipeline/AD/PX372696_02641791...,aicf_pg2,NaN,aicf_pg2,aicf_pg2,0.917460
2,doc-digitization-pipeline/AD/P0342280_02744692...,aicf_v1,"{'State': 'IL', 'Initial': 'R.', 'ZIP': '61234...",ACCIDENTAL INJURY CLAIM FORM Failure to comple...,doc-digitization-pipeline/AD/P0342280_02744692...,other,NaN,aicf_v1,aicf_v1,0.911419
3,doc-digitization-pipeline/AD/P0L700T5_02844807...,aicf_pg2,"{'No': 'X', '*First Name': 'M I C H A E L', 'D...",if you have additional bills or medical docume...,doc-digitization-pipeline/AD/P0L700T5_02844807...,aicf_pg2,NaN,aicf_pg2,aicf_pg2,0.931417
4,doc-digitization-pipeline/AD/P0P9L5Y4_02716028...,aicf_pg2,"{'""Date of Birth (mm/dd/yy)': 'D 8 / I 8 / 0 6...",02-10-21:02:42PM; AFLAC Claims ;7149869600 3/ ...,doc-digitization-pipeline/AD/P0P9L5Y4_02716028...,aicf_pg2,NaN,aicf_pg2,ptscf,0.781522


In [8]:
print(classification_report(list(filtered_df['revised_template']),list(filtered_df['results'])))

              precision    recall  f1-score   support

      acdbcf       1.00      1.00      1.00         1
       ahwcf       1.00      0.67      0.80         6
    ahwcf_v3       0.33      1.00      0.50         1
    ahwcf_v4       0.67      1.00      0.80         2
    aicf_pg1       1.00      0.99      0.99       200
    aicf_pg2       1.00      0.95      0.98       187
     aicf_v1       1.00      0.92      0.96        24
     aicf_v2       0.83      1.00      0.91         5
     aicf_v3       1.00      1.00      1.00         1
      canscr       1.00      1.00      1.00         1
      clmapp       1.00      1.00      1.00         3
    hicf_pg1       0.00      0.00      0.00         0
    hicf_pg2       0.88      1.00      0.93         7
 hicf_pg2_v2       0.00      0.00      0.00         1
 init_pg1_v2       1.00      1.00      1.00         1
    init_pg3       1.00      1.00      1.00         1
       other       1.00      0.67      0.80         3
     phystmt       0.00    

/opt/conda/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1

In [ ]:
count = 0
for n, row in filtered_df.iterrows():
    if row['revised_template'] != row['results']:
        count+=1
        print(n, row['png_path'])
print(f"\nCount: {count}")